In [1]:
%run "baby-jubjub.ipynb"

In [2]:
def commit(x, H):
    assert(0 < x < N.order())
    s = x % N.order()
    t = int(N.random_element(1, N.order() - 1))
    C = s*G + t*H
    return (C, t)

def open(C, t, x, H):
    assert(0 < x < N.order())
    s = x
    Cprime = s*G + t*H
    return C == Cprime

def addC(C1, C2):
    return C1 + C2

def addT(t1, t2):
    return t1 + t2 % N.order()

def addX(x1, x2):
    assert(0 < x1 + x2 < N.order())
    return x1 + x2

In [3]:
H = E(6194595726402067201576091900283561622215763079773074967268850179021717238447, 8439092241020226480363983149744167072619692543280365025111519919082806615610)
Hed = pointWToEd(H.xy(), F, Amont, Bmont)
Hed

(15062767207209273935069766228681029151541034596557745984629900633106846735903,
 6341322789198213736148938234331954507166091334711016221314502571133203693855)

In [4]:
class Account:
    def __init__(self, amount):
        self.amount = amount
        self.amount_in_bits = [amount >> i & 1 for i in range(32, -1, -1)]
        (self.commitment, self.blinding_factor) = commit(amount, H)
        self.blinding_factor_in_bits = [self.blinding_factor >> i & 1 for i in range(255, -1, -1)]

In [5]:
(a, b, c, d) = (Account(randint(0, 2**33)), Account(randint(0, 2**33)), Account(randint(0, 2**33)), Account(randint(0, 2**33)))

In [6]:
(a.amount, b.amount, c.amount, d.amount)

(6755660355, 957527168, 2340666061, 2679155564)

In [7]:
sumX = addX(addX(a.amount, b.amount), addX(c.amount, d.amount))
sumT = addT(addT(a.blinding_factor, b.blinding_factor), addT(c.blinding_factor, d.blinding_factor))
sumC = addC(addC(a.commitment, b.commitment), addC(c.commitment, d.commitment))

In [8]:
(open(a.commitment, a.blinding_factor, a.amount, H), 
 open(b.commitment, b.blinding_factor, b.amount, H), 
 open(c.commitment, c.blinding_factor, c.amount, H), 
 open(d.commitment, d.blinding_factor, d.amount, H),
 open(addC(a.commitment, b.commitment), addT(a.blinding_factor, b.blinding_factor), addX(a.amount, b.amount), H), 
 open(addC(c.commitment, d.commitment), addT(c.blinding_factor, d.blinding_factor), addX(c.amount, d.amount), H), 
 open(sumC, sumT, sumX, H))

(True, True, True, True, True, True, True)

In [9]:
import collections


def flatten(x):
    if isinstance(x, collections.Iterable):
        return [str(a) for i in x for a in flatten(i)]
    else:
        return [str(x)]

accounts = [a, b, c, d]

args = flatten([
    [pointWToEd(p.commitment.xy(), F, Amont, Bmont) for p in accounts],
    [p.amount_in_bits for p in accounts],
    [p.blinding_factor_in_bits for p in accounts],
    [p.amount for p in accounts],
    sumX
])

' '.join(args)

'415875841684742911533632413538711654786213925598584457141403615164527777037 5182627401359431743448843690711571842429225135186676329572872869640858769795 4788040944190702879485553848373438631354781394360910556041831104740910546651 13595836178491841617396194324501650920685545858362044956866301260898560303503 16903032901723492174378585153630193633667899743395361566166221483332461471016 21784722373952772412925627565977165569856829871927952108242913025705384404289 14244775406468778061318081514046218905987030480471049522887183628927988419775 17183959217100927814960654031713660060188019016455744414536182231933939614070 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0